# File Search Agent

### Connect to the Foundry

In [1]:
from util import get_project_client
project = get_project_client()

### Create Vector store

In [3]:
import os
uploaded_file_ids = []

#
try:
    for local_file in os.listdir("data"):
        # upload_and_poll ensures the upload is complete before continuing
        # "assistants" purpose tells the service these files are for agent usage
        upl = project.agents.files.upload_and_poll(
            file_path=os.path.join("data", local_file),
            purpose="assistants"
        )
        uploaded_file_ids.append(upl.id)
        print(f"✅ Uploaded: {local_file} -> File ID: {upl.id}")


            
    print(f"File IDs: {uploaded_file_ids}")

except FileNotFoundError:
    print(f"Error: The file '{local_file}' was not found.")
    raise

except Exception as e:
    print(f"An error occurred while processing the Excel file: {e}")
    raise


# Step 2: Create a vector store from the uploaded filesete
vs = project.agents.vector_stores.create_and_poll(
	file_ids=uploaded_file_ids,  # Pass all our uploaded file IDs
	name="form4-vector-store"               # Give our vector store a friendly name
)
print(f"🎉 Created vector store 'form4-vector-store', ID: {vs.id}")




✅ Uploaded: 0001062993-25-004143.pdf -> File ID: assistant-RPdQBQnZDvLggXYqcAHJi2
✅ Uploaded: 0001062993-25-004144.pdf -> File ID: assistant-TVRQXT3C92kohoMavwKqWV
✅ Uploaded: 0001062993-25-005509.pdf -> File ID: assistant-FQMtUDYYFqtbUDEPp4tUao
✅ Uploaded: 0001062993-25-005512.pdf -> File ID: assistant-JsCbrVVgiQKyJRKA1MdfcP
✅ Uploaded: 0001062993-25-005513.pdf -> File ID: assistant-CrDKTcqHGb2mxHD5xZT8Xx
✅ Uploaded: 0001062993-25-005514.pdf -> File ID: assistant-Rexc1QFfrxPjCDzqQEXusq
✅ Uploaded: 0001062993-25-005515.pdf -> File ID: assistant-JWxVDk3aKoTbzWKhsijTvy
✅ Uploaded: 0001062993-25-005517.pdf -> File ID: assistant-PGhHY2o91spLrqC5ymseKm
✅ Uploaded: 0001062993-25-005518.pdf -> File ID: assistant-9GnvZiB5dzc3XKtJaXeAkW
✅ Uploaded: 0001062993-25-005519.pdf -> File ID: assistant-NLvboxpbo1XQNjTJgNomag
File IDs: ['assistant-RPdQBQnZDvLggXYqcAHJi2', 'assistant-TVRQXT3C92kohoMavwKqWV', 'assistant-FQMtUDYYFqtbUDEPp4tUao', 'assistant-JsCbrVVgiQKyJRKA1MdfcP', 'assistant-CrDKTcqHGb2mxH

### Create Agent with FileSearchTool

In [5]:
import os 
from azure.ai.agents.models import FileSearchTool

# Create a file search tool
file_search = FileSearchTool(vector_store_ids=[vs.id])

# Create an agent with the file search tool
agent = project.agents.create_agent(
    model=os.environ["CHAT_MODEL"],  # Model deployment name
    name="fs-search-agent",  # Name of the agent
    instructions="""
        You are a helpful assistant with access to Form4 fillings.
        You:
        1. Always be polite and professional.
        2. Provide references to the files when possible
    """,
    tools=file_search.definitions,  # Tools available to the agent
    tool_resources=file_search.resources,  # Resources for the tools
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_0K8M8hPPZN7UidhKdhWzRJqe


### Create a thread

In [6]:
# Create a thread
thread = project.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

# Send a message to the thread
message = project.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="Who is Judson Altoff?",  # Message content
)
print(f"Created message, ID: {message['id']}")

Created thread, ID: thread_Fvo9JdJVBW6cFcJa5Zwah4PU
Created message, ID: msg_grZ49G1RYE79xfYcIkp8HYC3


In [7]:
run = project.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

Run finished with status: RunStatus.COMPLETED


In [11]:
messages = project.agents.messages.list(thread_id=thread.id, order="asc")
for msg in messages:
    role = msg.role
    content = msg.content[0].text.value
    print(f"{role.capitalize()}: {content}")

User: Who is Judson Altoff?
Assistant: Judson Altoff is identified in the Form 4 filing as an individual associated with Microsoft Corporation. Specifically, he is listed as the "EVP & Chief Business Officer" (Executive Vice President & Chief Business Officer) at Microsoft. This executive role indicates that he is a senior leader responsible for key aspects of Microsoft’s business operations, likely involving commercial strategy and partnerships .

If you need further details such as his share transactions or additional background information, please let me know!


### Util method message handeling

In [12]:
def sendMessage(msg):
    # Create a thread
    thread = project.agents.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Send a message to the thread
    message = project.agents.messages.create(
        thread_id=thread.id,
        role="user",
        content=msg,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

    run = project.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    messages = project.agents.messages.list(thread_id=thread.id, order="asc")
    for msg in messages:
        role = msg.role
        content = msg.content[0].text.value
        print(f"{role.capitalize()}: {content}")


In [13]:
sendMessage("Who is Hogan Kathleen?")


Created thread, ID: thread_ZLAx9SXzZwQumTkGuaFbK1JW
Created message, ID: msg_tyoHEj0UsdKgy1stjOL23TEE
Run finished with status: RunStatus.COMPLETED
User: Who is Hogan Kathleen?
Assistant: Hogan Kathleen is listed as a reporting person in the Form 4 filings, which typically means she is an executive or director at a publicly traded company, required to report transactions in company securities. In the data provided, Hogan Kathleen executed several transactions related to Microsoft Corporation stock, indicating she holds a significant role within the company. Most commonly, Kathleen Hogan is known as the Chief People Officer (Executive Vice President, Human Resources) at Microsoft Corporation.

If you need more specific details from the Form 4 itself, such as her exact title or more on her recent transactions, please let me know!


In [14]:
sendMessage("Each uploaded file is a form. Extract the name and the total number of stocks owned by the person. Output the results in a structured table format. The total number of stocks should be a numeric value.")

Created thread, ID: thread_CTKhzjnXHLJhbA52pkDlqm4B
Created message, ID: msg_jni4DLd2BNdPAOojXPxGP83d
Run finished with status: RunStatus.COMPLETED
User: Each uploaded file is a form. Extract the name and the total number of stocks owned by the person. Output the results in a structured table format. The total number of stocks should be a numeric value.
Assistant: Here is a structured table with the name of each reporting person and the total number of stocks (including both directly and indirectly owned common stocks and any derivative securities converted to common stock):

| Name                 | Total Stocks Owned | Source File                  |
|----------------------|-------------------|------------------------------|
| Reid Hoffman         | 31,681.302        | 0001062993-25-005509.pdf     |
| Hugh F. Johnston     | 8,793.645         | 0001062993-25-005512.pdf     |
| Teri List            | 25,263.17         | 0001062993-25-005513.pdf     |
| Penny S. Pritzker    | 25,094.463 